In [25]:
# 연습문제 : 청와대 국민청원 게시판 크롤링하기(url주소가 있는 경우)
# http://www1.president.go.kr/petitions

#Step 1. 필요한 모듈과 라이브러리를 로딩합니다.

from bs4 import BeautifulSoup
from selenium import webdriver
import time
import math
import numpy 
import pandas as pd  
import xlwt
import random
import os   # 폴더 생성하기 위해 필요함

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print("연습문제 : 청와대 국민청원 게시판 크롤링하기")
print("=" *100)

query_txt = '청와대국민청원'

cnt = int(input('1.크롤링 할 건수는 몇건입니까?: '))
page_cnt = math.ceil(cnt / 7)
print("크롤링 할 총 페이지 번호: ",page_cnt)


#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 ★★★★★★
f_dir = input("2.파일을 저장할 폴더명만 쓰세요(기본값:c:\\temp\\):")
if f_dir == '' :
    f_dir="c:\\Users\\"

# 저장될 파일위치와 이름을 지정합니다
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)

os.makedirs(f_dir+s+'-'+query_txt)
os.chdir(f_dir+s+'-'+query_txt)

ff_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.txt'
fc_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.csv'
fx_name=f_dir+s+'-'+query_txt+'\\'+s+'-'+query_txt+'.xls'
    
#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
chrome_path = "C:/Users/pc/chromedriver/chromedriver.exe"
driver = webdriver.Chrome(chrome_path)

driver.get('http://www1.president.go.kr/petitions')
driver.maximize_window() 

#안내창 닫기
try :
    driver.find_element_by_xpath('/html/body/div[1]/div/div[3]/table/tbody/tr/td[2]/div').click()
except :
    print('')
    
time.sleep(random.randrange(2,5))  # 2 - 5 초 사이에 랜덤으로 시간 선택

print("\n")

#Step 4. 각 게시글의 상세 정보를 수집합니다

click_count = 1

# 각 게시글의 URL 주소를 추출하여 목록을 먼저 만든 후 상세 크롤링을 진행합니다.
# URL 수집하여 목록 만들기
url_list=[]
count = 0
#while ( click_count <= page_cnt + 1)  :
for a in range(1,page_cnt+1) :
    
    print("%s 번째 페이지에서 게시글의 URL 정보를 추출합니다==============" %a)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    result_1 = soup.find_all('div','board text')
    result_2= result_1[2].find('div', class_='b_list category b_list2') 
    result_url_list=result_2.find('div','bl_body').find('ul','petition_list').find_all('li')
        
    for i in result_url_list:
        url = i.find('a')['href']
        print(url)
        full_url = 'http://www1.president.go.kr/'+ url
        url_list.append(full_url)
            
        count += 1

        if count == cnt :
            break  

    print("%s 번째 페이지까지 %s건 정보 수집 완료 ====================" %(a,count))
    print("\n")
    
    a += 1
    try :
        driver.find_element_by_link_text("%s" %a).click()
    except :
        driver.find_element_by_link_text('Next').click()
        
    time.sleep(2)

# Step 5. 각 게시글 별 상세 내역을 수집합니다
no = 1

cno2=[]
title2=[]
people2=[]
category2=[]
s_date2=[]
e_date2=[]
content2=[]

for i in range(0,len(url_list)):  
            
    print("\n")
    print("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no)

    driver.get(url_list[i])

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    content_all = soup.find('div','petitionsView_left_pg')

    f = open(ff_name, 'a',encoding='UTF-8')

    # 청원 등록 번호
    c_no = url_list[i].split('/')
    #cno_1 = c_no[4].split('?')
    cno_1 = c_no[5]        
    print('1.청원글번호: ',cno_1)
    f.write('\n')
    f.write("%s 번째 국민청원 게시글 상세 정보입니다==========================" %no + '\n')
    f.write('1.청원번호:' + cno_1 + '\n')
    cno2.append(cno_1)

    # 청원제목 추출
    title = content_all.find('h3','petitionsView_title').get_text()
    print('2.청원제목: ' , title)
    f.write('2.청원제목:' + title + '\n')
    title2.append(title)

    # 참여인원 추출
    people = content_all.find('h2','petitionsView_count').find('span','counter').get_text()
    print('3.참여인원: ' , people ,'명')
    f.write('3.참여인원:' + people + '\n')
    people2.append(people)

    # 카테고리
    cat_all = content_all.find('div','petitionsView_info').find('ul','petitionsView_info_list').find_all('li')

    category = cat_all[0].get_text().replace('카테고리','')
    print('4.카테고리: ',category)
    f.write('4.카테고리:' + category + '\n')
    category2.append(category)

    # 청원시작일
    s_date = cat_all[1].get_text().replace('청원시작','')
    print('5.청원시작일: ',s_date)
    f.write('5.청원시작일:' + s_date + '\n')
    s_date2.append(s_date)

    # 청원마감일
    e_date = cat_all[2].get_text().replace('청원마감','')
    print('6.청원마감일: ',e_date)
    f.write('6.청원종료일:' + e_date + '\n')
    e_date2.append(e_date)

    # 청원내용
    content = content_all.find('div','View_write').get_text().replace('\n','')
    print('7.청원내용: ', content.strip())
    f.write('7.청원내용:' + content.strip() + '\n')
    content2.append(content)

    f.close( )
    
    driver.back()  # 이전 페이지로 돌아가기 ★★★★★
    
    time.sleep(2)
    
    no += 1

    if no > cnt :
        break
            
# Step 6. 출력 결과를 저장하기
# 출력 결과를 표(데이터 프레임) 형태로 만들기

blue_house = pd.DataFrame()

blue_house['청원번호'] = cno2
blue_house['청원제목'] = pd.Series(title2) 
blue_house['참여자'] = pd.Series(people2)
blue_house['카테고리'] = pd.Series(category2)
blue_house['청원시작일'] = pd.Series(s_date2)
blue_house['청원종료일'] = pd.Series(e_date2)
blue_house['청원내용'] = pd.Series(content2)
       
# csv 형태로 저장하기
blue_house.to_csv(fc_name,encoding="utf-8-sig",index=False)

# 엑셀 형태로 저장하기
blue_house.to_excel(fx_name , index=False)

driver.close( )

연습문제 : 청와대 국민청원 게시판 크롤링하기
1.크롤링 할 건수는 몇건입니까?: 25
크롤링 할 총 페이지 번호:  4
2.파일을 저장할 폴더명만 쓰세요(기본값:c:\temp\):C:\Users\pc\Chap3.Crawling\


1 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/600507
/petitions/600506
/petitions/600505
/petitions/600504
/petitions/600503
/petitions/600502
/petitions/600501
1 번째 페이지까지 7건 정보 수집 완료 ====================


2 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/600500
/petitions/600499
/petitions/600498
/petitions/600497
/petitions/600496
/petitions/600495
/petitions/600494
2 번째 페이지까지 14건 정보 수집 완료 ====================


3 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/600493
/petitions/600492
/petitions/600491
/petitions/600490
/petitions/600489
/petitions/600488
/petitions/600487
3 번째 페이지까지 21건 정보 수집 완료 ====================


4 번째 페이지에서 게시글의 URL 정보를 추출합니다==============
/petitions/600486
/petitions/600485
/petitions/600484
/petitions/600483
4 번째 페이지까지 25건 정보 수집 완료 ====================




1 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글



5 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600503
2.청원제목:  [불공정거래 하도급법위반] 도움 부탁드립니다
3.참여인원:  212 명
4.카테고리:  기타
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  안녕하세요.저희는 대전에 있는 IoT 기반 안전관리솔루션을 전문으로 하는 중소기업입니다.오랜 고민 끝에, 하기와 같은 내용으로 도움을 부탁드립니다. 지난 일년간 대표인 저의 어리석은 판단으로 해당 건으로 현재까지 고통을 받고 있으며, 현재 많은 직원이 퇴사하고, 또 저희의 여러 협력 업체들도 일거리가 사라져 많은 어려움을 겪고 있습니다. 모쪼록 제대로 된 조사를 받을 수 있게 도움 부탁드립니다. 그리고, 하기 내용은 많은 내용 중 일부입니다. 작년 12월 부터, 추가적인 일들을 .. 아니 어려움을 계속 겪고 있습니다. 도움 부탁드립니다. 끝으로, 하기 내용은 사실 위주로 작성하다 보니 글 작성에 예의가 없음을 양해부탁드립니다.*****사건명 : BLE 기반 IoT 디바이스에 대한 지적재산권 침해 사건개요: - 저희는 대전에 있는 IoT 업체입니다(이하, 당사). 당사는 2018년 1월 국내에 있는 유명 반도체 업체에 “비상시 인원계수를 위한 BLE 기반 IoT 디바이스 납품 사업”의 입찰에 참여하였으며, 최종 납품 업체로 선정되어 이후 해당 제품을 현재까지 해당 반도체 업체에 8만 여개를 납품하고 있음- 해당 제품의 개발에 따른 인력 투입 및 연구비는 당사의 자체 비용으로 진행하였으며, 납품 계약은 제품의 대한 단품 납품 계약으로 진행함(지적재산권의 소유는 당사에 있음. 해당 제품에 대한 디자인 특허 및 펌웨어 등에 대한 소프트웨어 등록 완료)- 해당 반도체 업체에 영업을 전담하고 있는 같은 그룹의 계열사인 “S”사는 2020년 2월 해당 제품에 대한 듀얼(dual) 브랜드 정책을 당사에 고지함. 당사는 해당 IoT 디바이스에 대한 지적재산권을



9 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600499
2.청원제목:  2021년 민트세트
3.참여인원:  256 명
4.카테고리:  기타
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  2021년 민트세트가 나오는 8월11일, 많은 분들이 민트세트를 구매하려고 대기를 하였습니다.하지만 대기는 끊임없이 지속되었고 결국 구매를 못하시는 분들이 많아졌습니다.조폐공사 쇼핑몰에서는 민트세트가 나오는 날에는 렉을 방지시킬수 있는 방안을 찾거나 서버를 안정시킬수있는 방법을 마련해야한다고생각합니다.그리고 작년을 포함한 몇년간 50~100개 심지어 더 많은 민트세트를 구매하신 분들이있었습니다.이것은 조폐공사쇼핑몰에서 민트세트를 빼돌려서 지인에게 나눠줬다든지,아니면 어떠한 프로그램을 사용하여 대기열을 뚫는다던지 등등,공평하지 못한 방법으로 이루어졌을 것으로 생각이 듭니다.내년에는 이런일이 없도록 어떠한 조치를 마련해주었으면 좋겠습니다.


10 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600498
2.청원제목:  감염관리수당은 코로나 의료인력들을 위한 것이 맞습니까?
3.참여인원:  159 명
4.카테고리:  보건복지
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  국고로 지원한 코로나 감염관리수당에 대해 수도권 내 공공 의료원은 수당을 해당의료인력에게 공정하게 배분해줄 것을 정부에 요청드립니다. 현재 2021년 정부에서 지급된 "코로나 관련 의료인력 감염관리 수당 지원금" 은 의료인력을 위한 수당임을 수당명에 명시해놨음에도 불구하고 해당 의료기관 전 직원에게 1/N 로 행정지원금이라는 명목하에 지급되고 있습니다. 수당을 수령하는 사람 중 다수는 방호복을 입어보지도, 코로나환자를 대면해보지도 않은 부서들도, 관련 업무에서 아예 배제된 부서들도 대다수 포함되어 있습니다



15 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600493
2.청원제목:  9월 예비신부입니다. 방역수칙 모순점 개선부탁드립니다.
3.참여인원:  341 명
4.카테고리:  보건복지
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  결혼식 49인 인원제한은 도대체 어느 누구의 생각에서 나온 것인가요?되려 뷔페가 49인 제한이고 예식장이 면적당 인원제한해야하는 거 아닌가요?예식장들은 당연 뷔페는 (면적만 넓으면) 크게 제한없으니 보증인원은 줄여줄 수 없다고 말합니다.그럼 모든 피해는 그저 축복받아야 할 예비신랑, 신부들만 떠안습니다.예식장은 이러나 저러나 아무런 피해 없습니다.모든 예식장이 뷔페는 식당으로 들어가서 제한을 크게 안받고예식장만 제한이라 홀 & 로비 49인 제한해서 홀에 49인 들어가면 로비에는 어느 누구도 못 있는답니다.지인들이 축하해주러 와도 인사만 하고 바로 뷔페로 보내야하고그 마저도 단체사진 찍을 때 뷔페에서 밥먹는 사람들을 불러 올라오게 해서 로테이션식으로 촬영해야 한답니다.상식적으로 말이 된다고 생각하세요?본인들 결혼식에 그렇게 하라고 하면 흔쾌히 그렇게 해야죠! 하시겠어요?결혼식은 전국에서 모이는 개념이라 안된다고요?타지에서 오는 인원은 그렇게 많지않아요;그런 개념이면 모든 비행기, 기차, 지하철, 백화점, 아울렛 49인 제한해야하는 거 아니에요?전국, 전세계에서 이용하는데 그쵸?제한 하나요? 안하죠?왜 인생에 단 한번뿐인 예식장만 그렇게 제한하나요?모든 예비신랑, 신부들이 신혼여행도 남들이 갔던 해외로 가지 못하는 것도 서글픈데결혼식까지 고통을 줘야겠나요?그러면서 왜 종교 모임은 99인 제한인가요?형평성에 맞다고 생각하세요?종교모임 언제든지 할 수 있는 거 아니에요?1년에 한번 모이는 것도 아니잖아요?매주 모이는 거 아닌가요?매주 모이는 게 더 위험한거 아니에요?국민을 위한 정부 맞나요?아니면 종교계에 대선 눈치보고 그들을 위해 정



20 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600488
2.청원제목:  소상공인 죽이는 **대학교 갑질행정 막아주세요!!
3.참여인원:  177 명
4.카테고리:  행정
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  안녕하십니까, **대학교 **플라자  중식당 대표 ***,*** 입니다.이번에 저희 업체는 ** *대학교 *****의 갑질 행정으로 인해 정신적인 괴로움과 감당하기 힘든 금전적 손실을 보고 있습니다.저희처럼 힘없는 소상공인의 목소리는 누구도 들어주지 않았습니다.엄청난 정신적 고통과 막대한 금전적 피해만 저희에게 넘겨졌습니다. 이일에 관련해명확히 조사를 해야 하며 학교는 잘못된 행정 처리에 대한 책임을 져야한다고 생각합니다. 이번 사건으로 인해 **대학교에서 다시는 저희와 같은 일을 당하는 피해자가 없길 바랄 뿐입니다. 2019년 2월1일부터 2021년 1월 31일일 까지 갱신계약서를 작성했습니다.이후 매년1년 단위로 갱신한다. 라고 명시되어있습니다.(계약내용 변경사유등의 이유이며 만료시 어떤 이유에서라도 계약종료라고 명시되어있지 않습니다)2018년 이전에는 임대차 보호법으로 5년 보장이었으나 2018년 임대차 보호법 개정으로 인하여 2019년 갱신을 하였으므로 임대를 10년 보장 받을 수 있습니다.저희 업체는 5년간 운영하였고  유책사유가 없기에 5년 연장 가능한 상태입니다.학교측에서 협의 없이 일방적인 계약해지는 불가한 것으로 알고 있습니다.당사에 계약만료 전 연장계약불가라는 공문을 보낸 적이 없습니다.또 한 저희업장이 용도변경 된다는 공문 또한 보낸 적이 없습니다.1월14일  연장계약이 안 된다 업장을 정리하라는 통보를 들었습니다.운영을 계속 하겠다 의사를 밝혔으나 업장을 다른 용도로 사용 할 것이라는 통보를 받았습니다. 저희업체는 1월15일 연장계약희망 한다는 내용의 이메일을 보냈으나 문서로 온 답변은 없었습니다. 복지처장 *** 



23 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600485
2.청원제목:  소상공인 버팀목자금 플러스  부당한행정
3.참여인원:  119 명
4.카테고리:  행정
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  저는 서울에서 개인택시를 운행하고 있습니다너무나도 황당하고 도저히 납득할수없는 일이 발생하여서 도움을청합니다정부에서 어려운 소상공인에게 도움을 주려고 만든 4차 재난지원금(소상공인 버팀목자금 플러스)을 신청하였습니다매출이 전년도(2019년)보다 마이너스가 되었는데도 플러스가 되었다고 부지급이되었습니다이에 이의신청까지 했는데도 부지급이 되었습니다 이유인즉 너무나 어처구니가 없게도 2019년4월 30일 사업자등록을 하고 5월3일부터 영업을 시작했는데 (4월매출₩0) 4월30일 사업자등록을 했다는이유로 4월달 한달을 영업 한것으로 포함시켜 전년도보다 매출을 플러스로 만들었습니다4월달에 영업한것도 아니고 매출도 0원 인데도4월 한달을 영업한것으로 인정했다는것은 도저히 이해할수가 없는 행정이라고 생각합니다 정부에서는 사각지대가 없는 행정으로 어려운 소상공인에게 피해를 주지 않겠다고 공표를 했는데도 현실은 그렇치 않습니다진정 소상공인에게 도움을 주려고 만든 제도인지? 아님 보여주기식 행정인지? 벌써 수개월동안 희망 고문을 하고있습니다 벌써 5차재난지원금 소식까지 들리는 마당에 아직껏 이렇게 어려운 코로나 시기를 힘겹게 보내고 있습니다 이렇듯 틀에갇힌 불합리한 행정을 어떻게해야 구제를 받을수 있을까요


24 번째 국민청원 게시글 상세 정보입니다==========================
1.청원글번호:  600484
2.청원제목:  *** 출소 반대
3.참여인원:  113 명
4.카테고리:  기타
5.청원시작일:  2021-08-12
6.청원마감일:  2021-09-11
7.청원내용:  *** 출소 2022년으로 미뤄라


25 번째 국민청원 게시글 상세

<ipython-input-25-034bb732a60c>:204: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  blue_house.to_excel(fx_name , index=False)
